In [1]:
import torch
import torch.nn as nn
import numpy as np

# -------------------------
# Redefine the same model
# -------------------------
class FeatureTokenizer(nn.Module):
    def __init__(self, num_features, embed_dim):
        super(FeatureTokenizer, self).__init__()
        self.embed = nn.Linear(num_features, embed_dim)

    def forward(self, x):
        return self.embed(x)

class FTTransformer(nn.Module):
    def __init__(self, num_features, num_classes, embed_dim=64, num_heads=4, num_layers=2):
        super(FTTransformer, self).__init__()
        self.tokenizer = FeatureTokenizer(num_features, embed_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.tokenizer(x)  # Tokenize input features
        x = x.unsqueeze(1)     # Add sequence dimension
        x = self.transformer(x)
        x = x.squeeze(1)       # Remove sequence dimension
        x = self.fc(x)
        return x

# -------------------------
# Load the model
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = 20
num_classes = 7

model = FTTransformer(num_features=num_features, num_classes=num_classes).to(device)
model.load_state_dict(torch.load('./fttransf_new.pth', map_location=device))
model.eval()

# -------------------------
# Prepare new input sample
# -------------------------
# Example: 23 values as input (topsoil + subsoil + deepsoil + Soil as encoded feature)
new_input = np.array([
    25.3, 55.2, 6.5, 90, 40, 60,     # Topsoil
    14.1, 60.0, 6.4, 85, 38, 58,     # Subsoil
    23.5, 12.0, 5.3, 0, 0, 23,     # Deepsoil
    2, 5                                  # Encoded Soil and Crop
], dtype=np.float32)


input_tensor = torch.tensor(new_input).unsqueeze(0).to(device)  # shape [1, 23]

# -------------------------
# Predict
# -------------------------
with torch.no_grad():
    output = model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()
    print("Predicted Fertilizer Class Index:", predicted_class)

    # If you used LabelEncoder or custom mapping:
    fertilizer_labels = {
    0: "DAP and MOP",          # Source of Phosphorus and Potassium
    1: "Good NPK",             # No fertilizer needed
    2: "MOP",                  # Source of Potassium
    3: "Urea and DAP",         # Source of Nitrogen and Phosphorus
    4: "Urea and MOP",         # Source of Nitrogen and Potassium
    5: "Urea",                 # Source of Nitrogen only
    6: "DAP"                   # Source of Phosphorus only
}
    print("Predicted Fertilizer:", fertilizer_labels[predicted_class])

Predicted Fertilizer Class Index: 2
Predicted Fertilizer: MOP


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
